<body>
<h2>Project 5: Bias Variance Trade-off</h2>

<!--announcements-->
<blockquote>
    <center>
    <a href="http://blogs.worldbank.org/publicsphere/files/publicsphere/biased_processing.jpg"><img src="bias.jpg" width="600px" /></a>
    </center>
      <p><cite><center>"All of us show bias when it comes to what information we take in.<br>We typically focus on anything that agrees with the outcome we want."<br>
<b>--Noreena Hertz</b>
      </center></cite></p>
</blockquote>



<h3>Introduction</h3>

<p>
Recall that the squared error can be decomposed into <em>bias</em>, <em>variance</em> and <em>noise</em>: 
$$
    \underbrace{\mathbb{E}[(h_D(x) - y)^2]}_\mathrm{Error} = \underbrace{\mathbb{E}[(h_D(x)-\bar{h}(x))^2]}_\mathrm{Variance} + \underbrace{\mathbb{E}[(\bar{h}(x)-\bar{y}(x))^2]}_\mathrm{Bias} + \underbrace{\mathbb{E}[(\bar{y}(x)-y(x))^2]}_\mathrm{Noise}\nonumber
$$
We will now create a data set for which we can approximately compute this decomposition. 
The function <em><strong>toydata</strong></em> generates a binary data set with class $1$ and $2$. Both are sampled from Gaussian distributions:
$$
p(\vec x|y=1)\sim {\mathcal{N}}(0,{I}) \textrm { and } p(\vec x|y=2)\sim {\mathcal{N}}(\mu_2,{I}),
$$

where $\mu_2=[2;2]^\top$ (the global variable <em>OFFSET</em> $\!=\!2$ regulates these values: $\mu_2=[$<em>OFFSET</em> $;$ <em>OFFSET</em>$]^\top$).
</p>


<h3>Computing noise, bias and variance</h3>
<p>
You will need to edit four functions:  <em><strong>computeybar</strong></em>,  <em><strong>computehbar</strong></em>,  <em><strong>computevariance</strong></em> and  <em><strong>biasvariancedemo</strong></em>. First take a look at <strong>biasvariancedemo</strong> and make sure you understand where each function should be called and how they contribute to the Bias/Variance/Noise decomposition. <br/><br/>
</p>





**Libraries**: Before we get started we need to install a few libraries. You can do this by executing the following code. The first time you execute this code it may take a little while as it will install and compile the libraries on your computing node. 

In [ ]:
try
    using PyPlot
catch
    Pkg.add("PyPlot")
    Pkg.build("PyPlot")
end

**Toydata Helper Function**: Toydata is a helper function used to generate the the binary data with n/2 values in class 1 and n/2 values in class 2. With class 1 being the label for data drawn from a normal distribution having mean 0 and sigma 1. And clss 2 being the label for data drawn from a normal distribution with mean OFFSET and sigma 1.

In [ ]:
#<GRADED>

In [ ]:
function toydata(OFFSET,N)
    """ function (x,y)=toydata(OFFSET,N)
        This function constructs a binary data set. 
        Each class is distributed by a standard Gaussian distribution.
    
    INPUT: 
        OFFSET:  Class 1 has mean 0,  Class 2 has mean 0+OFFSET (in each dimension). 
        N: The function returns N data boints ceil(N/2) are of class 2, the rest of class 1
    """
    NHALF=Int(ceil(N/2));
    x=randn(N,2);
    x[NHALF:end,:]=x[NHALF:end,:]+OFFSET;
    y=ones(N);
    y[NHALF:end]=y[NHALF:end].*2;
    jj=randperm(N);
    x=x[jj,:];
    y=y[jj];
    return(x,y);
end;

In [ ]:
#</GRADED>

<p>
(a) <strong>Noise:</strong> First we focus on the noise. For this, you need to compute $\bar y(\vec x)$ in  <em><strong>computeybar</strong></em>. You can compute the probability $p(\vec x|y)$ with the equations $p(\vec x|y=1)\sim {\mathcal{N}}(0,{I}) \textrm { and } p(\vec x|y=2)\sim {\mathcal{N}}(\mu_2,{I})$. Then use Bayes rule to compute $p(y|\vec x)$. <br/><br/>
<strong>Note:</strong> You may want to use the function <em>normpdf</em>, which is defined for  you in <em><strong>computeybar</strong></em>.
<br/><br/></p>


In [ ]:
#<GRADED>

In [ ]:
function computeybar(xTe)
    """
    function [ybar]=computeybar(xTe);

    computes the expected label 'ybar' for a set of inputs x
    generated from two standard Normal distributions (one offset by OFFSET in
    both dimensions.)

    INPUT:
    xTe    : nx2 array of n vectors with 2 dimensions
    
    Globals:
    OFFSET : The OFFSET passed into the toyData function. The difference in the
             mu of labels class1 and class2 for toyData.

    OUTPUT:
    ybar   : a nx1 vector of the expected labels for vectors xTe
    """
    global OFFSET;
    n,temp=size(xTe);
    ybar=zeros(1,n);
    
    # Feel free to use the following function to compute p(x|y)
    normpdf(x,mu,σ)=exp(-0.5*((x-mu)./σ).^2)./(sqrt(2*pi) .* σ);
    
    # FILL IN CODE HERE
    throw("unimplemented")
    
    return(ybar[:])
end;

In [ ]:
#</GRADED>

**Visualizing the Data**:
You can now see the error of the bayes classifier. Below is a plotting of the two classes of points and the misclassified points.

In [ ]:
using PyPlot
global OFFSET;
OFFSET=2;
xTe,yTe=toydata(OFFSET,1000)

# compute Bayes Error
ybar=computeybar(xTe);
predictions=round(ybar);
errors=find(predictions.!=yTe);
err=length(errors)/length(yTe)*100;
println("Error of Bayes classifier: $err%.");

# plot data
i1=(yTe.==1.0);
i2=(yTe.==2.0);
clf();
plot(xTe[i1,1],xTe[i1,2],"r.");
hold(true);
plot(xTe[i2,1],xTe[i2,2],"b.");
plot(xTe[errors,1],xTe[errors,2],"ko",markersize=10,alpha=0.2);
title("Plot of data (misclassified points highlighted)");

<p>With the help of <strong>computeybar</strong> you can now compute the "noise" variable within <strong>biasvariancedemo</strong>. </p>

**kregression Helper Function**: 
<br/>
<strong>Important</strong> - $h_D$ is defined for you in <em><strong>kregression</strong></em>. It's kernelized ridge regression with kernel width $\sigma$ and regularization constant $\lambda$.
<br/><br/>

In [ ]:
#<GRADED>

In [ ]:
function kregression(xTr,yTr,σ=0.1,λ=0.01)
    """
    function kregression(xTr,yTr,sigma,lmbda)
    
    Input:
        xTr   | training data (nx2)
        yTr   | training labels (nx1)
        σ     | kernel width (>0)
        λ     | regularization constant (>0)
    
    Output:
        fun   | usage: predictions=fun(xTe);
    """
    function sqdistance(X,Z)
        n,d=size(X);
        m=size(Z,1);
        s1=sum(X.^2,2);
        s2=sum(Z.^2,2);
        D=broadcast(+,s1,broadcast(+,s2',-2.*X*Z'));
        return(D);
    end;
    kernel(x,z)=(1+sqdistance(x,z)./(2*σ^2)).^(-4);
    ridge(K,λ)=K+λ.*eye(size(K,1),size(K,2));
    beta=ridge(kernel(xTr,xTr),λ)\(yTr);
    fun(Xt)=kernel(Xt,xTr)*beta;
    return(fun);
end;

In [ ]:
#</GRADED>

<p>
(b) <strong>Bias:</strong> For the bias, you will need $\bar{h}$. Although we cannot compute the expected value  $\bar h\!=\!\mathbb{E}[h]$, we can approximate it by training many $h_D$ and averaging their predictions. Edit the file <em><strong>computehbar</strong></em>. Average over <em>NMODELS</em> different $h_D$, each trained on a different data set of <em>Nsmall</em> inputs drawn from the same distribution. Feel free to call <em><strong>toydata</strong></em> to obtain more data sets. <br/><br/>
</p>

In [ ]:
#<GRADED>

In [ ]:
function computehbar(xTe, sigma, lmbda)
    """
    function [hbar]=computehbar(xTe, sigma, lmbda ; Nsmall, NModel, OFFSET);

    computes the expected prediction of the average classifier (hbar)
    for data set xTe. 

    The training data of size Nsmall is drawn from toydata with OFFSET 
    with kernel regression with sigma and lmbda

    The "infinite" number of models is estimated as an average over NMODELS. 

    INPUT:
        xTe    | nx2 matrix, of n column-wise input vectors (each 2-dimensional)
        sigma  | kernel width of the RBF kernel
        lmbda  | regularization constant
    
    Global:
        NModel | Number of Models to average over
        OFFSET | The OFFSET passed into the toyData function. The difference in the
                 mu of labels class1 and class2 for toyData.
    
    OUTPUT:
        hbar   | nx1 vector with the predictions of hbar for each test input
    """
    global Nsmall,NMODELS,OFFSET
    n=size(xTe,1);
    hbar=zeros(n,1);
    for j=1:NMODELS
        # FILL IN CODES HERE
        throw("unimplemented")
    end;
    hbar=hbar./NMODELS;
    return(hbar)
end;

In [ ]:
#</GRADED>

<p>With the help of <strong>computehbar</strong> you can now compute the "bias" variable within <strong>biasvariancedemo</strong>. </p>


<p>(c) <strong>Variance:</strong> Finally, to compute the variance, we need to compute the term $\mathbb{E}[(h_D-\bar{h})^2]$. Once again, we can approximate this term by averaging over  <em>NMODELS</em> models. Edit the file <em><strong>computevariance</strong></em>. 
<br/></br></p>

In [ ]:
#<GRADED>

In [ ]:
function computevariance(xTe, sigma, lmbda, hbar)
    """
    function variance=computevariance(xTe,sigma,lmbda,hbar;Nsmall,NMODELS,OFFSET)

    computes the variance of classifiers trained on data sets from
    toydata.m with pre-specified "OFFSET" and 
    with kernel regression with sigma and lmbda
    evaluated on xTe. 
    the prediction of the average classifier is assumed to be stored in "hbar".

    The "infinite" number of models is estimated as an average over NMODELS. 

    INPUT:
    xTe       | nx2 matrix, of n column-wise input vectors (each 2-dimensional)
    sigma     | kernel width of the RBF kernel
    lmbda     | regularization constant
    hbar      | nx1 vector of the predictions of hbar on the inputs xTe
    
    Globals:
    Nsmall    | Number of samples drawn from toyData for one model
    NModel    | Number of Models to average over
    OFFSET    | The OFFSET passed into the toyData function. The difference in the
                mu of labels class1 and class2 for toyData.
    """
    global Nsmall,NMODELS,OFFSET;
    n=size(xTe,1);
    variance=zeros(n,1);
    for j=1:NMODELS
        # FILL IN CODES HERE
        throw("unimplemented")
    end;
    variance=mean(variance)/NMODELS;
    return(variance);
end;

In [ ]:
#</GRADED>

<p>With the help of <strong>computevariance</strong> you can now compute the "variance" variable within <strong>biasvariancedemo</strong>. </p>

<p>If you did everything correctly and call execute the following demo. You should see how the error decomposes (roughly) into bias, variance and noise when regularization constant $\lambda$ increases.</p>
<br/>

In [ ]:
# biasvariancedemo

using PyPlot

global Nsmall,Nbig,NMODELS,OFFSET;
Nsmall=10;   # how big is the training set size N
Nbig=10000;  # how big is a really big data set (approx. infinity)
NMODELS=100; # how many models do you want to average over
λs=-6:0.5:0; # What regularization constants to evaluate
σ=4;         # what is the kernel width?

# we store
Nlambdas  = length(λs);
lbias     = zeros(1,Nlambdas);
lvariance = zeros(1,Nlambdas);
ltotal    = zeros(1,Nlambdas);
lnoise    = zeros(1,Nlambdas);
lsum      = zeros(1,Nlambdas);


## Different regularization constant classifiers
for md=1:Nlambdas
    λ=2^λs[md];
    
    # use this data set as an approximation of the true test set
    xTe,yTe=toydata(OFFSET,Nbig);
    
    ## Estimate AVERAGE ERROR (TOTAL)
    total=0;
    for j=1:NMODELS
        xTr2,yTr2=toydata(OFFSET,Nsmall);        
        fsmall=kregression(xTr2,yTr2,σ,λ);
        total=total+mean((fsmall(xTe)-yTe).^2);
    end;
    total=mean(total)/NMODELS;

    
    ## Estimate Noise
    ybar=computeybar(xTe);
    noise=mean((yTe-ybar).^2);
    
    ## Estimating BIAS
    hbar=computehbar(xTe,σ, λ);
    bias=mean((hbar-ybar).^2);
    
    ## Estimating VARIANCE
    variance=computevariance(xTe,σ,λ,hbar);
        
    ## print and store results
    lbias[md]=bias;
    lvariance[md]=variance;
    ltotal[md]=total;
    lnoise[md]=noise;
    lsum[md]=lbias[md]+lvariance[md]+lnoise[md];
    @printf("Regularization λ=2^%2.1f: Bias: %2.4f Variance: %2.4f Noise: %2.4f Bias+Variance+Noise: %2.4f Test error: %2.4f\n",λs[md],lbias[md],lvariance[md],lnoise[md],lsum[md],ltotal[md]);
end;

In [ ]:
## plot results
figure()
plot(lbias[1:Nlambdas],"r-",linewidth=2);
plot(lvariance[1:Nlambdas],"k-",linewidth=2);
plot(lnoise[1:Nlambdas],"g-",linewidth=2);
plot(ltotal[1:Nlambdas],"b-",linewidth=2);
plot(lsum[1:Nlambdas],"k--",linewidth=2);

legend(["Bias","Variance","Noise","Test error","Bias+Var+Noise"]);
xlabel("Regularization \lambda=2^x",fontsize=18);
ylabel("Squared Error",fontsize=18);
xticks(1:Nlambdas,λs);


Feel free to modify $\lambda$/$\sigma$ in these two files. If you want the approximation to be more accurate, increase <em>NMODELS</em> and/or <em>Nbig</em> (the more models you train, the better your approximation will be for $\mathbb{E}[h]$ and $\mathbb{E}[(h_D-\bar{h})^2]$). 
You can also play around with the variable <em>Nsmall</em> which regulates how big your actual training is supposed to be. 
</p>


<h3>Note</h3>
<p>
When computing the bias and variance, you approximate the results by training many $h_D$. We set <em>NMODELS</em>=1000 and use some thresholds to test if your functions' results are correct. Unfortunately, as a result of this randomness, there is still a small chance that you will fail some test cases, even though your implementations are correct. <br/><br/>
If you can pass all the tests most of the times locally, then you are fine. In this case, if the autograder says your accuracy is not 100%, just commit the code again.<br/><br/>

There is no competition this time.
</p>